# import

In [1]:
import csv
import os
import sys
from concurrent.futures import ProcessPoolExecutor
from glob import glob, iglob

import numpy as np
import pandas as pd
import xmltodict
from tqdm import tqdm

<jemalloc>: MADV_DONTNEED does not work (memset will be used instead)
<jemalloc>: (This is the expected behaviour if you are running under QEMU)


In [2]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

# Read data

In [3]:
def safe_get(dictionary, key):
    value = dictionary.get(key, {})
    return value if type(value)==dict else {}

In [4]:
def recursive_get(dictionary, keys):
    for i, key in enumerate(keys):
        if i != len(keys) - 1:
            dictionary = safe_get(dictionary, key)
        else:
            dictionary = dictionary.get(key, {})
    return dictionary

In [5]:
years_path = glob("../data/NewYorkTimes/raw/*")

In [6]:
def parse_xml(root):
    dataset = pd.json_normalize(
        root,
        errors="ignore",
    )
    return_df = dataset.loc[: ,["nitf.head.meta", "nitf.head.pubdata.@date.publication"]]

    # full_text
    full_text = pd.DataFrame(
        root.get("nitf", {}).get("body", {}).get("body.content", {}).get("block", {})
    )
    value = full_text.loc[full_text["@class"] == "full_text", "p"].to_numpy()
    full_text_value = " ".join(value[0]) if type(value[0]) == list else value[0]
    return_df["full_text"] = full_text_value

    # classifier
    classifier_keys = ["nitf", "head", "docdata", "identified-content", "classifier"]
    classifier = pd.json_normalize(recursive_get(root, classifier_keys))
    classifier_value = classifier.loc[
        (classifier["@class"] == "online_producer")
        & (classifier["@type"] == "taxonomic_classifier"),
        "#text",
    ]
    classifier_value_joined = "%".join(classifier_value.tolist())
    return_df["classifier"] = classifier_value_joined

    return return_df

In [ ]:
dfs = []
for year_path in tqdm(years_path):
    dataset = []
    for file in iglob(f"{year_path}/*/*/*.xml"):
        with open(file, mode="r") as f:
            root = xmltodict.parse(f.read())
        try:
            dataset.append(parse_xml(root))
        except:
            continue
    df = pd.concat(dataset)
    df.to_csv(f"../data/NewYorkTimes/parsed/{year_path.split('/')[-1]}.csv")
    dfs.append(df)

 90%|█████████ | 19/21 [13:57:41<1:35:55, 2877.59s/it]

In [ ]:
dfs = pd.concat(dfs)

In [ ]:
dfs.isna().sum()

In [ ]:
dfs.to_csv("temp.csv")